## Installation and Imports

In [1]:
!pwd

/home/daniel/DARM/darm_mujoco/darm_training


In [2]:
# Configure env variables

# TODO: change path
import os
os.environ["DARM_MUJOCO_PATH"] = "/home/daniel/DARM/darm_mujoco"
os.getenv('DARM_MUJOCO_PATH')

'/home/daniel/DARM/darm_mujoco'

In [ ]:
# Check if GCC is installed
!gcc --version

In [ ]:
# Install GCC if absent
!sudo apt update
!sudo apt install build-essential -y

In [ ]:
# # Setup Mujoco for gym - If needed
# !apt-get install -y \
#     libgl1-mesa-dev \
#     libgl1-mesa-glx \
#     libglew-dev \
#     libosmesa6-dev \
#     software-properties-common

# !apt-get install -y patchelf

# !pip install gym

# !pip install free-mujoco-py

# import mujoco_py
# import gym

In [3]:
!pip install ray[rllib] torch
!pip install wandb
!pip install tensorflow_probability

In [4]:
%cd /home/daniel/DARM/darm_mujoco
!python setup.py install

/home/daniel/DARM/darm_mujoco
running install
/home/daniel/miniconda3/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/daniel/miniconda3/lib/python3.8/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
writing darm_gym_env.egg-info/PKG-INFO
writing dependency_links to darm_gym_env.egg-info/dependency_links.txt
writing requirements to darm_gym_env.egg-info/requires.txt
writing top-level names to darm_gym_env.egg-info/top_level.txt
reading manifest file 'darm_gym_env.egg-info/SOURCES.txt'
writing manifest file 'darm_gym_env.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/e

In [ ]:
# Check if mujoco import is successful
import mujoco

In [ ]:
# If mujoco import fails, update pandas and restart runtime
!pip install pandas -U

In [3]:
import ray
from ray.rllib.algorithms.es import ESConfig
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback

import gym
from darm_gym_env import DARMSFEnv

## Register Environment with RLlib

In [4]:
# env_creator = lambda env_config: gym.make("darm/DarmSFHand-v0", render_mode=None, hand_name="hand1") # DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1") # 

def make_env(env_config):
    env = gym.wrappers.TimeLimit(env=DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1"), max_episode_steps=100)
    return env
env_creator = lambda env_config: make_env(env_config) #gym.wrappers.TimeLimit(env=DARMSFEnv(render_mode=None, reaction_time=0.08, hand_name="hand1"), max_episode_steps=100)

register_env("darm/DarmSFHand-v0", env_creator)

## Configure and Run

In [5]:
# TODO:
# change: rollout_workers
# change: gpu

config = (
    ESConfig()
    .environment(
        env="darm/DarmSFHand-v0"
    )
    .rollouts(
        num_rollout_workers=3,
        num_envs_per_worker=16,
        rollout_fragment_length=1,
        recreate_failed_workers=True,
        num_consecutive_worker_failures_tolerance=10,
        restart_failed_sub_environments=True,
    )
    .resources(num_gpus=0)
    # .evaluation(evaluation_interval=100) # For 1000 timesteps iter; 100 evals
    .framework(framework="torch")
)
# config.to_dict()

In [6]:
# TODO:
# change: rollout_workers
# change: gpu
# change: tags
# change: name

wandb_init = dict(
    save_code=True,
    resume=True,
    config={
        "env": "DARMSFHand-v0",
        "num_rollout_workers": 3,
        "num_envs_per_worker": 16,
        "recreate_failed_workers": True,
        "num_consecutive_worker_failures_tolerance": 10,
        "restart_failed_sub_environments": True,
        "num_gpus": 0,
        "framework": "torch"
    },
    tags=["single_finger", "es", "delta_target", "test", "local", "no_vel_penalty", "effort_penalty"],
    notes="Fixed the env to use targets that are delta increaments from the starting state. Removed velocity penalty, and used only effort penalty",
    name="ES_Test_DARMSF_DELTA_TARGET"
    # job_type=
    # monitor_gym=
)

In [7]:
%cd /home/daniel/DARM/darm-mujoco/darm_training/

[Errno 2] No such file or directory: '/home/daniel/DARM/darm-mujoco/darm_training/'
/home/daniel/DARM/darm_mujoco/darm_training


In [8]:
# TODO: 
# change: name
# change: checkpoint_freq

sync_config = tune.SyncConfig()

tuner = tune.Tuner(
    "ES",
    run_config=air.RunConfig(
        name="ES_Test_DARMSF_DELTA_TARGET",
        local_dir=f"{os.getenv('DARM_MUJOCO_PATH')}/darm_training/results",
        sync_config=sync_config,
        stop={"training_iteration": 10_000, "episode_reward_mean": 200},
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True,
            checkpoint_score_attribute="episode_reward_mean",  # or leave to save last chkpts
            checkpoint_score_order="max",
            checkpoint_frequency=1,  #50,
            num_to_keep=3
        ),
        callbacks=[
                WandbLoggerCallback(project="DARM", 
                                    api_key="392c8a47eb0658eb5c71190757a69110e2140f4a",
                                    save_checkpoints=True, 
                                    **wandb_init)
            ],
        ),
    param_space=config.to_dict()
)

results = tuner.fit()

2023-02-13 02:06:06,579	INFO worker.py:1538 -- Started a local Ray instance.


wandb: Currently logged in as: danieladejumo. Use `wandb login --relogin` to force relogin
(ES pid=13633) 2023-02-13 02:06:11,816	WARNING algorithm_config.py:488 -- Cannot create ESConfig from given `config_dict`! Property __stdout_file__ not supported.
(ES pid=13633) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(ES pid=13633)   logger.warn(
(ES pid=13633) 2023-02-13 02:06:13,329	INFO policy.py:1147 -- Policy (worker=local) running on CPU.
(ES pid=13633) 2023-02-13 02:06:13,329	INFO torch_policy.py:184 -- Found 0 visible cuda devices.
(ES pid=13633) 2023-02-13 02:06:13,339	INFO es.py:396 -- Creating shared noise table.


(ES pid=13633) Loaded XML file successfully


(raylet) [2023-02-13 02:06:16,470 E 13372 13417] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-13_02-06-04_495000_13264 is over 95% full, available space: 80138240; capacity: 31845081088. Object creation will fail if spilling is required.


(ES pid=13633) 2023-02-13 02:06:25,853	INFO es.py:401 -- Creating actors.
(ES pid=13633) 2023-02-13 02:06:25,886	INFO trainable.py:172 -- Trainable.setup took 13.669 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(ES pid=13633) 2023-02-13 02:06:25,886	WARNING util.py:66 -- Install gputil for GPU system monitoring.
(ES pid=13633) 2023-02-13 02:06:25,897	INFO es.py:568 -- Collected 0 episodes 0 timesteps so far this iter
(raylet) [2023-02-13 02:06:26,476 E 13372 13417] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-13_02-06-04_495000_13264 is over 95% full, available space: 75501568; capacity: 31845081088. Object creation will fail if spilling is required.


...

(Worker pid=13853) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(Worker pid=13853)   logger.warn(
(Worker pid=13853) 2023-02-13 02:06:32,073	INFO policy.py:1147 -- Policy (worker=local) running on CPU.
(Worker pid=13853) 2023-02-13 02:06:32,073	INFO torch_policy.py:184 -- Found 0 visible cuda devices.
(Worker pid=13853) /home/daniel/miniconda3/lib/python3.8/site-packages/ray/rllib/algorithms/es/es_torch_policy.py:36: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525491394/work/torch/csrc/utils/tensor_numpy.cpp:199.)
(Worker pid=13853)   

(Worker pid=13852) Loaded XML file successfully
(Worker pid=13851) Loaded XML file successfully


(Worker pid=13851) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(Worker pid=13851)   logger.warn(
(Worker pid=13851) 2023-02-13 02:06:33,114	INFO policy.py:1147 -- Policy (worker=local) running on CPU.
(Worker pid=13851) 2023-02-13 02:06:33,115	INFO torch_policy.py:184 -- Found 0 visible cuda devices.
(Worker pid=13851) /home/daniel/miniconda3/lib/python3.8/site-packages/ray/rllib/algorithms/es/es_torch_policy.py:36: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525491394/work/torch/csrc/utils/tensor_numpy.cpp:199.)
(Worker pid=13851)   

Trial name,date,done,episode_len_mean,episode_reward_mean,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,perf,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_this_iter,timesteps_total,training_iteration,trial_id,warmup_time
ES_darm_DarmSFHand-v0_98e9e_00000,2023-02-13_02-22-34,False,100,-121.689,,861b20d418d04ecc8a21ec4945e5dc64,Daniel,"{'weights_norm': 547.98346, 'grad_norm': 5.518588, 'update_ratio': 0.04050255, 'episodes_this_iter': 1000, 'episodes_so_far': 8014}",8,192.168.84.35,"{'cpu_util_percent': 87.5747572815534, 'ram_util_percent': 78.61504854368934}",13633,968.775,145.21,968.775,1676251354,794272,99235,794272,8,98e9e_00000,13.6759


wandb: Adding directory to artifact (/home/daniel/DARM/darm_mujoco/darm_training/results/ES_Test_DARMSF_DELTA_TARGET/ES_darm_DarmSFHand-v0_98e9e_00000_0_2023-02-13_02-06-08/checkpoint_000001)... Done. 0.0s
(ES pid=13633) 2023-02-13 02:08:25,044	INFO es.py:568 -- Collected 0 episodes 0 timesteps so far this iter
(ES pid=13633) 2023-02-13 02:08:25,804	INFO es.py:568 -- Collected 6 episodes 600 timesteps so far this iter
(ES pid=13633) 2023-02-13 02:08:26,434	INFO es.py:568 -- Collected 12 episodes 1101 timesteps so far this iter
(raylet) [2023-02-13 02:08:26,578 E 13372 13417] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-13_02-06-04_495000_13264 is over 95% full, available space: 74313728; capacity: 31845081088. Object creation will fail if spilling is required.
(ES pid=13633) 2023-02-13 02:08:26,985	INFO es.py:568 -- Collected 18 episodes 1602 timesteps so far this iter
(ES pid=13633) 2023-02-13 02:08:27,691	INFO es.py:568 -- Collected 24 episodes 2202 timesteps so far 

In [16]:
# Ensure wandb is sysncing to cloud
# cd to darm_training again if not

In [6]:
# TODO:
# change: experiment name

# Restore Interrupted run
tuner = tune.Tuner.restore(
    f"{os.getenv('DARM_MUJOCO_PATH')}/darm_training/results/SF_rllib_es_vast",
    resume_errored=True
)
tuner

In [7]:
results = tuner.get_results()
results

In [8]:
# Get the best result based on a particular metric.
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")
best_result

Result(metrics={'episode_reward_mean': 266.62735, 'episode_len_mean': 100.0, 'timesteps_this_iter': 120910, 'info': {'weights_norm': 598.2583, 'grad_norm': 4.5833297, 'update_ratio': 0.025018256, 'episodes_this_iter': 1212, 'episodes_so_far': 47290}, 'done': True, 'trial_id': 'f5823_00000', 'perf': {'cpu_util_percent': 59.660000000000004, 'ram_util_percent': 19.2}, 'experiment_tag': '0'}, error=None, log_dir=PosixPath('/home/daniel/DARM/darm_mujoco/darm_training/results/SF_rllib_es_vast/ES_darm_DarmSFHand-v0_f5823_00000_0_2023-02-13_01-37-21'))

In [9]:
# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint
best_checkpoint

Checkpoint(local_path=/home/daniel/DARM/darm_mujoco/darm_training/results/SF_rllib_es_vast/ES_darm_DarmSFHand-v0_f5823_00000_0_2023-02-13_01-37-21/checkpoint_000039)

In [10]:
# Get Algorithm from saved checkpoint
# from ray.rllib.algorithms.algorithm import Algorithm
# algo = Algorithm.from_checkpoint(best_checkpoint._local_path)
# algo

algo = config.build()
algo.restore(best_checkpoint._local_path)

Loaded XML file successfully


/home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2023-02-13 03:48:45,085	INFO worker.py:1538 -- Started a local Ray instance.
(raylet) [2023-02-13 03:48:55,065 E 21016 21044] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-13_03-48-42_978782_20905 is over 95% full, available space: 103149568; capacity: 31845081088. Object creation will fail if spilling is required.
2023-02-13 03:49:01,286	INFO es.py:401 -- Creating actors.
2023-02-13 03:49:01,523	INFO trainable.py:172 -- Trainable.setup took 19.468 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-02-13 03:49:01,526	WARNING util.py:66 -- Install gputil for GPU system monitoring.
2023-02-13 03:49:01,544	INFO filter_manager.py:34 -- Synchronizing filters ...
(raylet) [2023-02-13 03:49:05,071 E 21016 21044] (raylet) file_system_monitor.cc:105:

(Worker pid=21290) Loaded XML file successfully
(Worker pid=21288) Loaded XML file successfully
(Worker pid=21289) Loaded XML file successfully


(Worker pid=21289) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(Worker pid=21289)   logger.warn(
(Worker pid=21289) 2023-02-13 03:49:08,513	INFO policy.py:1147 -- Policy (worker=local) running on CPU.
(Worker pid=21289) 2023-02-13 03:49:08,513	INFO torch_policy.py:184 -- Found 0 visible cuda devices.
(raylet) [2023-02-13 03:49:15,083 E 21016 21044] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-13_03-48-42_978782_20905 is over 95% full, available space: 103096320; capacity: 31845081088. Object creation will fail if spilling is required.


In [11]:
# resume the interrupted run
tuner.fit()

2023-02-11 17:26:26,421	INFO trial_runner.py:688 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2023-02-11 17:26:26,422	INFO trial_runner.py:825 -- Using following checkpoint to resume: /home/daniel/DARM/darm_mujoco/darm_training/results/Test_DARMSF_DELTA_TARGET/experiment_state-2023-02-11_17-23-28.json
2023-02-11 17:26:26,426	WARNING trial_runner.py:830 -- Attempting to resume experiment from /home/daniel/DARM/darm_mujoco/darm_training/results/Test_DARMSF_DELTA_TARGET. This will ignore any new changes to the specification.
2023-02-11 17:26:26,440	INFO tune.py:653 -- TrialRunner resumed, ignoring new add_experiment but updating trial resources.


(raylet) [2023-02-11 17:26:26,618 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1061683200; capacity: 31845081088. Object creation will fail if spilling is required.
wandb: Currently logged in as: danieladejumo. Use `wandb login --relogin` to force relogin
(SAC pid=15703) 2023-02-11 17:26:30,934	WARNING algorithm_config.py:488 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
(SAC pid=15703) 2023-02-11 17:26:31,413	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(raylet) [2023-02-11 17:26:36,627 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1061359616; capacity: 31845081088. Object creation will fail if spilling is required.


(RolloutWorker pid=15846) Loaded XML file successfully
(RolloutWorker pid=15844) Loaded XML file successfully


(RolloutWorker pid=15846) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=15846)   logger.warn(
(RolloutWorker pid=15844) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=15844)   logger.warn(
(RolloutWorker pid=15844) 2023-02-11 17:26:38,825	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


(RolloutWorker pid=15845) Loaded XML file successfully


(RolloutWorker pid=15845) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=15845)   logger.warn(
(SAC pid=15703) /home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(SAC pid=15703)   logger.warn(
(SAC pid=15703) 2023-02-11 17:26:40,232	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(SAC pid=15703) 2023-02-11 17:26:40,261	WARNING util.py:66 -- Install gputil for GPU system monitoring.


(SAC pid=15703) Loaded XML file successfully


(SAC pid=15703) 2023-02-11 17:26:40,442	INFO trainable.py:790 -- Restored on 192.168.152.36 from checkpoint: /tmp/checkpoint_tmp_7f50b6e15e2c473dba807bf1d398566d
(SAC pid=15703) 2023-02-11 17:26:40,442	INFO trainable.py:799 -- Current state after restoring: {'_iteration': 11, '_timesteps_total': None, '_time_total': 113.04964661598206, '_episodes_total': 114}
(SAC pid=15703) 2023-02-11 17:26:40,721	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
(SAC pid=15703) 2023-02-11 17:26:40,721	WARNING deprecation.py:47 -- DeprecationWarning: `concat_samples` has been deprecated. Use `concat_samples() from rllib.policy.sample_batch` instead. This will raise an error in the future!
(raylet) [2023-02-11 17:26:46,634 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, avai

wandb: Adding directory to artifact (/home/daniel/DARM/darm_mujoco/darm_training/results/Test_DARMSF_DELTA_TARGET/SAC_darm_DarmSFHand-v0_6a944_00000_0_2023-02-11_17-23-28/checkpoint_000012)... Done. 0.0s
(raylet) [2023-02-11 17:27:36,665 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1055997952; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 17:27:46,672 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1055973376; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 17:27:56,678 E 14732 14777] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-11_17-23-24_266872_14581 is over 95% full, available space: 1055977472; capacity: 31845081088. Object creation will fail if spilling is required.
(raylet) [2023-02-11 17:2

In [21]:
env = gym.make("darm/DarmSFHand-v0", render_mode="human", hand_name="hand1")

obs = env.reset()

episode_reward = 0
done = False

while not done:
    env.render()
    action = algo.compute_single_action(obs)
    obs,rew, done, info = env.step(action)
    episode_reward += rew
    # print(f"Step Reward: {rew}, \n Action: {action} \n Info: {info} \n\n\n")

env.close()

Loaded XML file successfully


In [22]:
episode_reward

-130.76633861660957

(raylet) [2023-02-13 03:50:45,247 E 21016 21044] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-02-13_03-48-42_978782_20905 is over 95% full, available space: 103010304; capacity: 31845081088. Object creation will fail if spilling is required.
